In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


Для задачи классификации выживаемости на Титанике буду использовать метрику macro **F-меру**. Т.к. она балансирует precision и recal. Нам важно найти баланс между ложноположительными и ложноотрицательными значениями. Кроме того, F-мера лучше отрабатывает на несбалансированных данных в отличии от precision и recal. Macro позволяет учесть выживших и невыживших в равной степени, не привязываясь к несбалансированости в датасете

In [3]:
data.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Dooley, Mr. Patrick",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


## Предобработка данных
Конвертирую колонку 'Sex' и 'Embarked' в числовой формат. Остальные категориальные признаки удалю, т.к. они имеют большую вариативность

In [4]:
data_preproc = pd.concat([data.select_dtypes(include='number').drop(['Survived'], axis='columns'), pd.get_dummies(data[['Sex', 'Embarked']], dtype = 'int')], axis='columns')
data_preproc.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,1,0,0


проверим пропущенные значения и заполним их средним значением

In [5]:
data_preproc.isna().sum()

,0
PassengerId,0
Pclass,0
Age,177
SibSp,0
Parch,0
Fare,0
Sex_female,0
Sex_male,0
Embarked_C,0
Embarked_Q,0


In [6]:
data_preproc.fillna(data_preproc.mean(), inplace=True)

т.к. все признаки находятся в разном числовом диапазоне, то применим норализацию ко всем данным

In [7]:
scaler = MinMaxScaler()
data_norm = pd.DataFrame(scaler.fit_transform(data_preproc), columns = data_preproc.columns)
data_norm.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.000000,1.0,0.271174,0.125,0.0,0.014151,0.0,1.0,0.0,0.0,1.0
1,0.001124,0.0,0.472229,0.125,0.0,0.139136,1.0,0.0,1.0,0.0,0.0


In [8]:
#разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    data_norm, data['Survived'], test_size=0.3, random_state=42, stratify=data['Survived'])

In [9]:
print('объем обучающей выборки: ', len(y_train))
print('объем тестовой выборки: ', len(y_test))

объем обучающей выборки:  623
объем тестовой выборки:  268


# Baseline

In [10]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)
print(f1_score(y_test, y_pred_dummy, average='macro'))

0.3810623556581986


# Logistic Regression

In [11]:
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print(f1_score(y_test, y_pred_lr, average='macro'))

0.7890547263681592


# SVM

In [12]:
svc_clf = SVC(kernel='linear', random_state=42)
svc_clf.fit(X_train, y_train)
y_pred_svc = svc_clf.predict(X_test)
print(f1_score(y_test, y_pred_svc, average='macro'))

0.7607142857142857


#Выводы
Логистическая регрессия показывает наилучший результат - 78,9% классификации выживаемости. Это почти в 2 раза лучше работы случайного классификатора. Метод  опорных векторов показал точность на 2% ниже. Это может говорить о линейной разделимости в данных. Также это может быть связано с малым количеством данных для эффективного обучения SVM.